In [131]:
#Import necessary libraries
import pandas as pd
import calendar
from datetime import datetime
import holidays
import category_encoders as ce
from sklearn.linear_model import LinearRegression


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#Prediction imports
import requests
import json
from datetime import datetime 
import pytz

In [86]:
#load in the data
df = pd.read_csv('csv_files/toll_weather.csv')
df = df.drop_duplicates()

/home/slittle/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
#Categorize the weather description as none, light, medium, or heavy
df['Weather Desc'] = df['Weather Desc'].fillna('none')
df['Weather'] = df['Weather Desc'].map({'none':'none', 'Thunderstorm':'heavy', 
 'Light thunderstorm, rain':'med', 'Heavy thunderstorm, rain':'heavy',
 'Mist':'light', 'In the vicinity thunderstorm':'light',
 'In the vicinity thunderstorm, light rain':'light', 'Light rain, mist':'light',
 'Thunderstorm, rain, mist':'heavy', 'Light rain':'light', 'Rain':'med',
 'Heavy thunderstorm, rain, mist':'heavy', 'Light thunderstorm, rain, mist':'med',
 'Heavy rain, mist':'heavy', 'Rain, mist':'med', 'Heavy rain':'heavy', 'Light drizzle, mist':'light',
 'In the vicinity thunderstorm, rain':'med', 'Thunderstorm, rain':'heavy', 'Haze':'none',
 'In the vicinity thunderstorm, light rain, mist':'light', 'Light drizzle':'light',
 'In the vicinity thunderstorm, heavy rain, mist':'heavy',
 'In the vicinity thunderstorm, heavy rain':'heavy',
 'In the vicinity thunderstorm, rain, mist':'med', 'Ground fog':'light',
 'Light rain, snow, mist':'heavy', 'Light rain, ice pellets':'heavy',
 'Light rain, snow, ice pellets, mist':'heavy', 'Light rain, ice pellets, mist':'heavy',
 'Light rain, snow':'med', 'Light snow':'heavy', 'Light snow, mist':'heavy',
 'Light ice pellets, rain':'heavy',
 'Light funnel cloud(s) (tornado or water-spout), rain':'heavy', 'Snow, mist':'heavy',
 'Light snow, ice pellets, mist':'heavy', 'Snow, ice pellets, mist':'heavy',
 'Light funnel cloud(s) (tornado or water-spout), mist':'heavy',
 'Light ice pellets, mist':'heavy', 'Ice pellets, rain':'heavy'})

In [88]:
#Drop features that do not correlate
df = df.drop(['Wind Direction', 'Cloud Cover', 'Unnamed: 0', 'Visibility', 'Pressure', 'Weather Desc'], axis=1)
df['Wind Speed'] = df['Wind Speed'].fillna(3.8)
#df = pd.get_dummies(data=df, columns=['Weather Desc'])

In [89]:
#Add ML useable columns to represent the date and time
df["date"] = df.StartDateTime.apply(lambda x : x.split()[0])
df["hour"] = df.StartDateTime.apply(lambda x : x.split()[1].split(":")[0])
df["minute"] = df.StartDateTime.apply(lambda x : x.split()[1].split(":")[1])
df["weekday"] = df.date.apply(lambda dateString : calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])
df["month"] = df.date.apply(lambda dateString : calendar.month_name[datetime.strptime(dateString,"%Y-%m-%d").month])
df["year"] = df.date.apply(lambda dateString : int(dateString[:4]) - 2017)

#Add a feature that is a 1 if the date is a federal holiday, 0 otherwise
us_holidays = holidays.UnitedStates()
df['Holiday'] = df.date.apply(lambda dateString : int(dateString in us_holidays))

#One hot encode categorical columns
encoder = ce.one_hot.OneHotEncoder(cols = ['weekday', 'month', 'Direction', 'Weather'])
df = encoder.fit_transform(df)



In [90]:
#Drop december 2018, as there was a government shutdown that month and it skews our data
df = df[(df.month_10 == 0) & (df.year == 1)]


In [91]:
#determine whether morning tolling is currently active
def morning_tolling(datetime):
    hour = int(datetime[11:13])
    minute = int(datetime[14:16])
    if (hour == 9 and minute >= 30) or (hour == 13 and minute <= 30) or (hour >= 10 and hour <= 12):
        return 1
    return 0

In [92]:
#determine whether evening tolling is currently active
def evening_tolling(datetime):
    hour = int(datetime[11:13])
    if (hour >= 19 and hour <= 23):
        return 1
    return 0

In [93]:
#Add two columns to specify whether morning tolling or evening tolling is active
df['Morning Tolling'] = df['StartDateTime'].apply(lambda x: morning_tolling(x))
df['Evening Tolling'] = df['StartDateTime'].apply(lambda x: evening_tolling(x))

In [94]:
#Drop columns we no longer need
df = df.drop(columns=['StartDateTime', 'EndDateTime', 'date'])

In [95]:
pd.set_option('display.max_columns', 100)
df.head()

,EndZoneID,StartZoneID,TollRate,Direction_1,Direction_2,Temperature,Humidity,Wind Speed,Weather_1,Weather_2,Weather_3,Weather_4,hour,minute,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year,Holiday,Morning Tolling,Evening Tolling
0,3120,3100,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,18,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,3130,3100,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,18,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,3230,3210,0.0,0,1,24.0,57.0,4.0,1,0,0,0,15,18,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,3110,3100,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,18,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,3100,3100,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,18,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


For each combination of start/end zone, add the past hour of tolls as 10 feature columns

Dataframe is in chronological order
want to organize it in terms of each route
then when we have these groups, add the past 10 tolls to each row

In [96]:
routes = []
zones = [[3100, 3110, 3120, 3130], [3200, 3210, 3220, 3230]]
for direction in zones:
    i = 0
    for zone in direction:
        start_zone = df[df.StartZoneID == zone]
        j = i
        while j < 4:
            new_route = start_zone[start_zone.EndZoneID == direction[j]]
            routes.append(new_route)
            j +=1
        i += 1

In [97]:
for route in routes:
    tolls = route.TollRate
    for i in range(2):
        route['Toll_Prev_{ID}'.format(ID=str(i + 1))] = route['TollRate'].shift((i + 1), fill_value = 0)

In [98]:
routes[5].head()

,EndZoneID,StartZoneID,TollRate,Direction_1,Direction_2,Temperature,Humidity,Wind Speed,Weather_1,Weather_2,Weather_3,Weather_4,hour,minute,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year,Holiday,Morning Tolling,Evening Tolling,Toll_Prev_1,Toll_Prev_2
12,3120,3110,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,18,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0
31,3120,3110,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,24,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0
33,3120,3110,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,24,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0
80,3120,3110,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,30,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0
87,3120,3110,0.0,1,0,24.0,57.0,4.0,1,0,0,0,15,30,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0


In [99]:
concat = pd.concat(routes)

In [100]:
from sklearn.model_selection import train_test_split
y = concat['TollRate']
X = concat.drop(columns=['TollRate'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [102]:
model = LinearRegression()

#model = model.fit(X_train, y_train)
#pred = model.predict(X_test)

model = model.fit(X, y)

In [104]:
#from sklearn.metrics import mean_absolute_error
#mean_absolute_error(pred, y_test)

Ingest new data in order to make prediction:
    need current weather API
    Make rest of data look the same:
        run holiday calculator thing
        change datetime
        etc.
    need counts of cars in the past 5, 10, 20, etc minutes
   
   
Pull this info every 6 minutes (count time on python), whenever new toll data comes online

In [105]:
url = 'https://smarterroads.org/dataset/download/29?token=rroAyo8V7jUoB4R7og4o9zBkagSL2JBQmbrrdKt4j1qIFzKBUBFMaZhHFm1FeI0b&file=hub_data/TollingTripPricing-I66/TollingTripPricing_current.xml'

In [139]:
import wget
wget.download(url, './current_toll/cur_toll')

'./current_toll/cur_toll'

In [107]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import csv

rows = []
path = './current_toll/cur_toll'
root = ET.parse(path).getroot()
for elem in root:
    tag = {}if __name__ == '__main__':
    tag["EndZoneID"] = elem.attrib['EndZoneID']
    tag["StartZoneID"] = elem.attrib['StartZoneID']
    tag["StartDateTime"] = elem.attrib['IntervalDateTime']
    tag["EndDateTime"] = elem.attrib['IntervalEndDateTime']
    tag["TollRate"] = elem.attrib['ZoneTollRate']
    tag["Direction"] = elem.attrib['CorridorID']
    rows.append(tag)

In [108]:
dfNew = pd.DataFrame(rows)


In [109]:
dfNew['Direction'] = dfNew['Direction'].astype(int)
dfNew['StartZoneID'] = dfNew['StartZoneID'].astype(int)
dfNew['EndZoneID'] = dfNew['EndZoneID'].astype(int)
dfNew['TollRate'] = dfNew['TollRate'].astype(float)

In [110]:
routes = []
zones = [[3100, 3110, 3120, 3130], [3200, 3210, 3220, 3230]]
for direction in zones:
    i = 0
    for zone in direction:
        start_zone = dfNew[dfNew.StartZoneID == zone]
        j = i
        while j < 4:
            new_route = start_zone[start_zone.EndZoneID == direction[j]]
            routes.append(new_route)
            j +=1
        i += 1

In [111]:
for route in routes:
    route['Toll_Prev_2'] = route['TollRate'].shift((-1), fill_value = 0)

In [112]:
dfNew = pd.concat(routes)
dfNew = dfNew.rename(columns={'TollRate':'Toll_Prev_1'})


In [113]:
#take only the columns with the most current toll information
recent_time = dfNew['StartDateTime'].iloc[0]
dfNew = dfNew[dfNew['StartDateTime'] == recent_time]


In [114]:
#Get current time to make predictions on
tz = pytz.timezone("GMT")
time = datetime.now().astimezone(tz)

In [115]:
#Add ML useable columns to represent the date and time
length = dfNew.shape[0] * 2

dfNew["hour"] = pd.Series([time.hour] * length)
dfNew["minute"] = pd.Series([time.minute] * length)
dfNew["weekday"] = pd.Series([calendar.day_name[time.weekday()]] * length)
dfNew["month"] = pd.Series([calendar.month_name[time.month]] * length)
dfNew["year"] = pd.Series([time.year - 2017] * length)

#Add a feature that is a 1 if the date is a federal holiday, 0 otherwise
us_holidays = holidays.UnitedStates()
dfNew['Holiday'] = pd.Series([int(str(time) in us_holidays)] * length)

In [116]:
def cur_morning_tolling():
    hour = time.hour
    minute = time.minute
    if (hour == 9 and minute >= 30) or (hour == 13 and minute <= 30) or (hour >= 10 and hour <= 12):
        return 1
    return 0

In [117]:
def cur_evening_tolling():
    hour = time.hour
    if (hour >= 19 and hour <= 23):
        return 1
    return 0

In [118]:
#Add two columns to specify whether morning tolling or evening tolling is active
dfNew['Morning Tolling'] = pd.Series([cur_morning_tolling()] * length)
dfNew['Evening Tolling'] = pd.Series([cur_evening_tolling()] * length)

In [119]:
#Drop columns we no longer need
dfNew = dfNew.drop(columns=['StartDateTime', 'EndDateTime'])

In [66]:
def get_cur_weather(dfNew):
    url = 'https://api.weatherbit.io/v2.0/current?city=Arlington,VA&key=62c667044ed34c21941755b53b286186'
    weather_json = requests.post(url = url)
    weather_dict = json.loads(weather_json.text)
    weather_data = weather_dict['data'][0]

    #need wind speed, temp, humidity, and how heavy the weather is
    wind_speed = weather_data['wind_spd']
    temp = weather_data['temp']
    humidity = weather_data['rh']
    precipitation = float(weather_data['precip'])
    desc = 'none'
    if (float(weather_data['snow']) > 0) or (precipitation > 7.5):
        desc = 'heavy'
    elif precipitation > 5:
        desc = 'med'
    elif precipitation > 2:
        desc = 'light'

    dfNew['Temperature'] = pd.Series([temp] * length)
    dfNew['Wind Speed'] = pd.Series([wind_speed] * length)
    dfNew['Weather'] = pd.Series([desc] * length)
    dfNew['Humidity'] = pd.Series([humidity] * length)
    
    return dfNew

In [134]:
def get_weather_dict():
    url = 'https://api.weatherbit.io/v2.0/forecast/hourly?city=Arlington,VA&key=62c667044ed34c21941755b53b286186'
    weather_json = requests.post(url = url)
    weather_dict = json.loads(weather_json.text)
    return weather_dict

def get_future_weather(dfNew, weather_dict, hour):
    #Get just the data from the necessary hour
    weather_data = weather_dict['data'][hour]

    #need wind speed, temp, humidity, and how heavy the weather is
    wind_speed = weather_data['wind_spd']
    temp = weather_data['temp']
    humidity = weather_data['rh']
    precipitation = float(weather_data['precip'])
    desc = 0
    if (float(weather_data['snow']) > 0) or (precipitation > 7.5):
        desc = 3
    elif precipitation > 5:
        desc = 2
    elif precipitation > 2:
        desc = 1

    dfNew['Temperature'] = pd.Series([temp] * length)
    dfNew['Wind Speed'] = pd.Series([wind_speed] * length)
    dfNew['Humidity'] = pd.Series([humidity] * length)
    
    cols = ['Weather_1', 'Weather_2', 'Weather_3', 'Weather_4']:
    for col in cols:
        dfNew[col] = pd.Series([0]*length)
    if desc == 0:
        dfNew['Weather_1'] = pd.Series([humidity] * length)
    elif desc == 1:
    
    elif desc == 2:
        
    elif desc == 3


    
    return dfNew

In [120]:
get_cur_weather(dfNew)

,Direction,EndZoneID,StartZoneID,Toll_Prev_1,Toll_Prev_2,hour,minute,weekday,month,year,Holiday,Morning Tolling,Evening Tolling,Temperature,Wind Speed,Weather,Humidity
3,1100,3100,3100,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
2,1100,3110,3100,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
1,1100,3120,3100,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
0,1100,3130,3100,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
10,1100,3110,3110,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
9,1100,3120,3110,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
8,1100,3130,3110,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
15,1100,3120,3120,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
14,1100,3130,3120,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38
18,1100,3130,3130,0.00,0.00,23,0,Tuesday,July,2,0,0,1,32.2,1.34,none,38


In [121]:
#One hot encode categorical columns
### USE CATEGORICAL ENCODERS (from LendingClub project)
dfNew['Direction'] = dfNew['Direction'].map({1100:'E', 1200:'W'})
dfNew = encoder.fit_transform(dfNew)

In [122]:
all_columns = ['EndZoneID', 'StartZoneID', 'Direction_1', 'Direction_2',
       'Temperature', 'Humidity', 'Wind Speed', 'Weather_1', 'Weather_2',
       'Weather_3', 'Weather_4', 'hour', 'minute', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'year', 'Holiday', 'Morning Tolling', 'Evening Tolling', 'Toll_Prev_1',
       'Toll_Prev_2']
cur_columns = dfNew.columns
needed_columns = list(set(all_columns) - set(cur_columns))

In [123]:
for item in needed_columns:
    dfNew[item] = pd.Series([0] * length)

In [124]:
dfNew = dfNew[all_columns]

In [47]:
dfNew.shape

(20, 38)

In [55]:
dfNew = dfNew.drop(columns=['Prediction'])

In [49]:
pred

array([ 0.02053494,  0.02722375,  0.03391257,  0.04060138,  0.02096282,
        0.02765164,  0.03434045,  0.0213907 ,  0.02807952,  0.02181859,
        5.93725273,  9.89215557, 10.14560777, 10.86444909,  3.71681022,
        4.46378917,  5.18263049,  1.00284096,  1.47491891,  0.50974209])

In [129]:
#dfNew['Prediction'] = pred
#dfNew
dfNew

,EndZoneID,StartZoneID,Direction_1,Direction_2,Temperature,Humidity,Wind Speed,Weather_1,Weather_2,Weather_3,Weather_4,hour,minute,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year,Holiday,Morning Tolling,Evening Tolling,Toll_Prev_1,Toll_Prev_2
3,3100,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
2,3110,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
1,3120,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
0,3130,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
10,3110,3110,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
9,3120,3110,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
8,3130,3110,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
15,3120,3120,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
14,3130,3120,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
18,3130,3130,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00


In [136]:
predictions = []
cur_df = dfNew.copy()
pred = model.predict(dfNew)

In [137]:
weather_dict = get_weather_dict()
hour_counter = 0
for i in range(20):
    print('here')
    #Make prediction on current dataframe:
    pred = model.predict(cur_df)
    predictions.append(pred)
    
    #Move the previous tolls up one slot
    cur_df['Toll_Prev_2'] = cur_df['Toll_Prev_1']
    cur_df['Toll_Prev_1'] = pred
    
    #Increase the time by X minutes
    time_inc = 3
    hour = cur_df['hour'].iloc[0]
    minute = cur_df['minute'].iloc[0]
    if minute + time_inc > 60:
        minute = minute + time_inc - 60
        if hour + 1 > 23:
            hour = 0
        else:
            hour +=1
        cur_df = get_future_weather(cur_df, weather_dict, hour_counter)
        hour_counter += 1
        print(cur_df)
    else:
        minute += 6
    cur_df['minute'] = pd.Series([minute] * length)
    cur_df['hour'] = pd.Series([hour] * length)



here
here
here
here
here
here
here
here
here
here
here
    EndZoneID  StartZoneID  Direction_1  Direction_2  Temperature  Humidity  \
3        3100         3100            1            0         29.6        52   
2        3110         3100            1            0         29.6        52   
1        3120         3100            1            0         29.6        52   
0        3130         3100            1            0         29.6        52   
10       3110         3110            1            0         29.6        52   
9        3120         3110            1            0         29.6        52   
8        3130         3110            1            0         29.6        52   
15       3120         3120            1            0         29.6        52   
14       3130         3120            1            0         29.6        52   
18       3130         3130            1            0         29.6        52   
23       3200         3200            0            1         29.6        52 

ValueError: could not convert string to float: 'none'

In [138]:
for pred in predictions:
    print(pred[10])

2.96919445449814
2.938160337714457
2.9086442488313207
2.880726284110186
2.8543904778083187
2.829615591347781
2.806380358346098
2.7846637840731683
2.764445159037613
2.7457040560966934
2.7284203266637888


In [133]:
cur_df

,EndZoneID,StartZoneID,Direction_1,Direction_2,Temperature,Humidity,Wind Speed,Weather_1,Weather_2,Weather_3,Weather_4,hour,minute,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year,Holiday,Morning Tolling,Evening Tolling,Toll_Prev_1,Toll_Prev_2
3,3100,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
2,3110,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
1,3120,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
0,3130,3100,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
10,3110,3110,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
9,3120,3110,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
8,3130,3110,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
15,3120,3120,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
14,3130,3120,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
18,3130,3130,1,0,32.2,38,1.34,1,0,0,0,23,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.00,0.00
